In [1]:
from kickstart import load_data

dataset = load_data("mlm_items_train.jsonlines")

In [2]:
print dataset[0].keys()

[u'seller_id', u'top_level_category', u'title', u'price', u'site', u'buying_mode', u'seller_power_status', u'sold_quantity', u'category_id', u'id', u'category_path']


In [3]:
print dataset[0].values()

[75259727, u'MLM1648', u'Ventilador De Aspas Eva Usb Para Laptop Y Pc Flexible', 39, u'MLM', u'buy_it_now', u'platinum', 21, u'MLM54184', u'MLM505628392', [u'MLM1648', u'MLM1712', u'MLM10345', u'MLM54184']]


In [4]:
from random import shuffle

shuffle(dataset)
print dataset[0].values()

[890912, u'MLM1747', u'Eliminador De Ruidos De Punterias Liqui Moly', 200, u'MLM', u'buy_it_now', u'gold', 7, u'MLM5324', u'MLM504870116', [u'MLM1747', u'MLM3172', u'MLM5324']]


Filtrar y remover features que no voy a necesitar.

In [5]:
import pandas as pd
import numpy as np

columns = [u'seller_id', u'top_level_category', u'price']
df = pd.DataFrame(dataset)
df.drop([label for label in dataset[0].keys() if label not in columns], axis=1, inplace=True)

df.seller_id = df.seller_id.astype(str)

df.ix[0]

price                     200
seller_id              890912
top_level_category    MLM1747
Name: 0, dtype: object

Transformamos a una lista de diccionarios para poder vectorizar.

In [6]:
df_d = df.T.to_dict().values()
df_d[0]

{u'price': 200.0, u'seller_id': '890912', u'top_level_category': u'MLM1747'}

Ahora vectorizamos el set.

In [10]:
from sklearn.feature_extraction import DictVectorizer

vec = DictVectorizer()
df_dv = vec.fit_transform(df_d)

df_dv[0]

<1x20412 sparse matrix of type '<type 'numpy.float64'>'
	with 3 stored elements in Compressed Sparse Row format>

In [6]:
from collections import Counter

for_sale = Counter(x['seller_id'] for x in train)
vectorize = lambda datum: (datum['price'], for_sale[datum['seller_id']])

In [7]:
train_len = int(len(train)*0.9)
train_split = train[:train_len]
test_split = train[train_len:]

print len(train_split)
print len(test_split)

90000
10000


Generar el train set de inputs.

In [8]:
X_train = [vectorize(x) for x in train_split]

Generar el train set de outputs.

In [9]:
#from sklearn.preprocessing import LabelEncoder

#enc = LabelEncoder()
#y_train = enc.fit_transform(pd.DataFrame(train_split)['top_level_category'].tolist())

y_train = [x["top_level_category"] for x in train_split]

De igual manera con los tests sets.

In [10]:
#X_test = pd.DataFrame(test_split).as_matrix(columns).astype(np.float)
#y_test = enc.fit_transform(pd.DataFrame(test_split)['top_level_category'].tolist())

X_test = [vectorize(x) for x in test_split]
y_test = [x["top_level_category"] for x in test_split]

Entrenamos el clasificador elegido.

In [13]:
from sklearn.preprocessing import OneHotEncoder

train_enc = OneHotEncoder()
y_train = train_enc.fit_transform(y_train)

test_enc = OneHotEncoder()
y_test = test_enc.fit_transform(y_test)

ValueError: invalid literal for long() with base 10: 'MLM1648'

In [ ]:
y_train[0]
y_test[0]

In [11]:
#from sklearn.tree import DecisionTreeClassifier
#classifier = DecisionTreeClassifier()

from sklearn.linear_model import LinearRegression
classifier = LinearRegression(n_jobs=2)

In [12]:
classifier.fit(X_train, y_train)

TypeError: cannot perform reduce with flexible type

In [48]:
prediction = classifier.predict(X_test)

from random import randint

nidx = randint(0, len(prediction))
print enc.inverse_transform([int(prediction[nidx])])
print enc.inverse_transform([y_test[nidx]])

[u'MLM1367']
[u'MLM1648']


In [49]:
sames = sum(1 for y1, y2 in zip(y_test, prediction) 
            if enc.inverse_transform([int(y1)]) == enc.inverse_transform([int(y2)]))
print sames*1./len(y_test)

0.0085


In [27]:
def calc_score(y_test, prediction):
    sames = sum(1 for y1, y2 in zip(y_test, prediction) if y1 == y2)
    return sames*1./len(y_test)

In [28]:
print calc_score(y_test, prediction)

0.0


Vamos a probar cross val score, para variar el slice de test. KFold cross val. Por ejemplo si cortamos en slices de 10% para validar, es 10-fold cv.

In [24]:
from sklearn.cross_validation import cross_val_score

scores = cross_val_score(classifier, X_train, y=y_train, cv=10, n_jobs=2)
print scores.mean()

0.455790653357
